In [15]:
from __future__ import division
from nltk.book import *
from __future__ import division
import nltk
import numpy
import matplotlib
import nltk, re, pprint
from urllib import urlopen
import sys
#reload(sys)                       <----- activate this for reading 
#sys.setdefaultencoding('utf8')

from bs4 import BeautifulSoup